In [1]:
#importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 6
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#classes = dataset.classes
#print(classes)

In [2]:
#defining class for CNN
class FlexibleCNN(nn.Module):
    def __init__(self, num_classes, in_channels=3, conv_filters=[16, 32, 64, 128, 256], 
                 kernel_sizes=[3, 3, 3, 3, 3], pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=512, activation='ReLU', batch_norm = True, dropout = 0.0):
        self.conv_filters = conv_filters
        self.kernel_sizes = kernel_sizes
        self.pool_sizes = pool_sizes
        self.dense_units = dense_units
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        
        super(FlexibleCNN, self).__init__()

        #convolutional layers
        self.conv_layers = nn.ModuleList()
        prev_out_size = 224
        in_channels = in_channels
        for out_channels, kernel_size, pool_size in zip(conv_filters, kernel_sizes, pool_sizes):
            conv_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size),
                self.get_activation(activation),
                nn.MaxPool2d(pool_size)
            )
            
            
            if batch_norm:
                conv_layer.add_module(f"batch_norm_{out_channels}", nn.BatchNorm2d(out_channels))
            if dropout > 0.0:
                conv_layer.add_module(f"dropout_{out_channels}", nn.Dropout2d(dropout))
            self.conv_layers.append(conv_layer)
            in_channels = out_channels
            
        def cal_size(stride, padding, kernel_size, prev_size):
            new_size = (prev_size-kernel_size+2*padding)/stride + 1
            return new_size//2
        
        prev_size = 224
        
        for i in range(5):
            new_size1 = cal_size(stride = 1, padding = 0, kernel_size = self.kernel_sizes[i], prev_size= prev_size)
            prev_size = new_size1
        print(new_size1)

        #fully connected layers
        #self.fc1 = nn.Linear(conv_filters[-1] * 7 * 7, dense_units)
        self.fc1 = nn.Linear(self.conv_filters[4] * int(new_size1) * int(new_size1), self.dense_units)
        self.fc2 = nn.Linear(self.dense_units, 10)#num_classes)

    def forward(self, x):
        # Forward pass through convolutional layers
        for conv_layer in self.conv_layers:
            x = conv_layer(x)

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)

        # Forward pass through fully connected layers
        #x = F.relu(self.fc1(x))
        x = self.get_activation(self.activation)(self.fc1(x))
        x = self.fc2(x)
        return x

    def get_activation(self, activation):
        if activation == 'ReLU':
            return nn.ReLU()
        elif activation == 'GELU':
            return nn.GELU()
        elif activation == 'SiLU':
            return nn.SiLU()
        elif activation == 'Mish':
            return nn.Mish()
        else:
            raise ValueError("Invalid activation function")
            
#model = FlexibleCNN(10)
#print(model)

In [3]:
!pip install wandb

In [4]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [5]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test23',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3]]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'batch_norm':{
            'values': [True,False]
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': [False]
        },
        'num_dense':{
            'values': [128, 256]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

Create sweep with ID: sc97klsv
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2/sweeps/sc97klsv


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=wandb.config.activation).to(device)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            total_train = 0
            correct_train = 0
            running_loss = 0
            # Train the model
            model.train()
            for images, labels in train_loader:
                
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # Calculate the training accuracy
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                
            train_accuracy = 100*correct_train/total_train
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%')
            wandb.log({'train_loss': round(running_loss / len(train_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in val_loader:
                    
                    images = images.to(device)
                    labels = labels.to(device)
            
                    # Forward pass
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
        
                val_accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'val_accuracy':val_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training')

wandb.agent(sweep_id, function=main,count=20) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: 6oaz8uw1 with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: True
wandb: 	data_augment: False
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [3, 5, 5, 7, 7]
wandb: 	num_dense: 128
wandb: Currently logged in as: prabhat-kumar. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


1.0
Epoch [1/6], Training Accuracy: 22.54%
Epoch [1/6], Validation Accuracy: 25.05%
Epoch [2/6], Training Accuracy: 28.25%
Epoch [2/6], Validation Accuracy: 29.55%
Epoch [3/6], Training Accuracy: 31.79%
Epoch [3/6], Validation Accuracy: 31.40%
Epoch [4/6], Training Accuracy: 34.39%
Epoch [4/6], Validation Accuracy: 32.05%
Epoch [5/6], Training Accuracy: 35.82%
Epoch [5/6], Validation Accuracy: 34.05%
Epoch [6/6], Training Accuracy: 38.83%
Epoch [6/6], Validation Accuracy: 33.75%
Finished Training


epoch,▁▂▄▅▇█
train_accuracy,▁▃▅▆▇█
train_loss,█▆▄▃▂▁
val_accuracy,▁▅▆▆██
epoch,6
train_accuracy,38.83
train_loss,1.76
val_accuracy,33.75


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: z51qn0cz with config:
wandb: 	activation: SiLU
wandb: 	batch_norm: False
wandb: 	data_augment: False
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	num_dense: 128
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


5.0
Epoch [1/6], Training Accuracy: 26.70%
Epoch [1/6], Validation Accuracy: 29.10%
Epoch [2/6], Training Accuracy: 34.59%
Epoch [2/6], Validation Accuracy: 32.40%
Epoch [3/6], Training Accuracy: 38.60%
Epoch [3/6], Validation Accuracy: 35.70%
Epoch [4/6], Training Accuracy: 43.12%
Epoch [4/6], Validation Accuracy: 32.80%
Epoch [5/6], Training Accuracy: 49.19%
Epoch [5/6], Validation Accuracy: 36.15%
Epoch [6/6], Training Accuracy: 55.13%
Epoch [6/6], Validation Accuracy: 33.30%
Finished Training


...

wandb: Agent Starting Run: xpozael7 with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: False
wandb: 	data_augment: False
wandb: 	dropout: 0.3
wandb: 	filt_org: [32, 64, 128, 256, 512]
wandb: 	kernel_size: [7, 7, 5, 5, 3]
wandb: 	num_dense: 256
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


3.0
Epoch [1/6], Training Accuracy: 16.88%
Epoch [1/6], Validation Accuracy: 19.00%
Epoch [2/6], Training Accuracy: 23.19%
Epoch [2/6], Validation Accuracy: 25.65%
Epoch [3/6], Training Accuracy: 26.49%
Epoch [3/6], Validation Accuracy: 25.70%
Epoch [4/6], Training Accuracy: 29.15%
Epoch [4/6], Validation Accuracy: 27.55%
Epoch [5/6], Training Accuracy: 30.49%
Epoch [5/6], Validation Accuracy: 30.25%
Epoch [6/6], Training Accuracy: 33.09%
Epoch [6/6], Validation Accuracy: 25.95%
Finished Training


epoch,▁▂▄▅▇█
train_accuracy,▁▄▅▆▇█
train_loss,█▄▃▂▂▁
val_accuracy,▁▅▅▆█▅
epoch,6
train_accuracy,33.09
train_loss,1.89
val_accuracy,25.95


wandb: Agent Starting Run: 93xmvy1h with config:
wandb: 	activation: GELU
wandb: 	batch_norm: True
wandb: 	data_augment: False
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 64, 128, 256, 512]
wandb: 	kernel_size: [3, 5, 5, 7, 7]
wandb: 	num_dense: 256
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


1.0
Epoch [1/6], Training Accuracy: 22.20%
Epoch [1/6], Validation Accuracy: 28.25%
Epoch [2/6], Training Accuracy: 28.35%
Epoch [2/6], Validation Accuracy: 30.75%
Epoch [3/6], Training Accuracy: 31.88%
Epoch [3/6], Validation Accuracy: 31.30%
Epoch [4/6], Training Accuracy: 33.73%
Epoch [4/6], Validation Accuracy: 33.30%


## Testing using best hyperparameter

In [ ]:
#importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 6
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = ImageFolder(root='/kaggle/input/i-naturalist/inaturalist_12K/val', transform=transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#classes = dataset.classes
#print(classes)

In [ ]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test(test_data)',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3]]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'batch_norm':{
            'values': [True,False]
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': [False]
        },
        'num_dense':{
            'values': [128, 256]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=wandb.config.activation).to(device)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            total_train = 0
            correct_train = 0
            running_loss = 0
            # Train the model
            model.train()
            for images, labels in train_loader:
                
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # Calculate the training accuracy
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                
            train_accuracy = 100*correct_train/total_train
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%')
            wandb.log({'train_loss': round(running_loss / len(train_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # test the model
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    
                    images = images.to(device)
                    labels = labels.to(device)
            
                    # Forward pass
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
        
                test_accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.2f}%')
                wandb.log({'test_accuracy':test_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training')

wandb.agent(sweep_id, function=main,count=10) # calls main function for count number of times.
wandb.finish()